<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

from utilities import read_csv

def get_combined_signal(stock):
    # Fetch monthly stock data
    data = yf.download(stock + '.NS', period='5y', interval='1mo')

    data['Symbol'] = stock
    # Calculate 5-period and 13-period EMA
    data['5_EMA'] = data['Close'].ewm(span=5, adjust=False).mean()
    data['13_EMA'] = data['Close'].ewm(span=13, adjust=False).mean()

    # Calculate MACD (8-period EMA - 21-period EMA) and Signal line (9-period EMA of MACD)
    data['26_EMA'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['12_EMA'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['MACD'] = data['12_EMA'] - data['26_EMA']
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Determine if MACD is up or down
    data['MACD_Up'] = data['MACD'] > data['Signal_Line']
    data['MACD_Down'] = data['MACD'] < data['Signal_Line']

    # Determine combined signal
    data['Momentm'] = 'NA'
    data.loc[(data['5_EMA'] > data['13_EMA']) & (data['MACD_Up']), 'Momentm'] = 'UP'
    data.loc[(data['5_EMA'] < data['13_EMA']) & (data['MACD_Down']), 'Momentm'] = 'DOWN'

    return data[['Symbol', 'Close', 'Momentm']]

mypps_df = read_csv('myProspectsScrips.csv')

exclude = ['CADILAHC','MMTC', 'SCHAEFFLER ']

stock_ids = mypps_df[~mypps_df['Symbol'].isin(exclude) ]['Symbol'].unique()

def get_mypps_momentum(stock_id):
  df_prec_dev = pd.DataFrame()
  for stock_id in stock_ids:
    df_pos = get_combined_signal(stock_id)

    df_pos['Prev_Momentm'] = df_pos['Momentm'].shift(1)
    df_pos = df_pos[df_pos['Momentm'] != df_pos['Prev_Momentm']]
    df_pos['Signal']= 'Sell'
    df_pos.loc[df_pos['Momentm'] == 'UP', 'Signal'] = 'Buy'
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop(['Prev_Momentm', 'Prev_Signal'], axis=1, inplace=True)
    df_pos['Close'] = round(df_pos['Close'],0)
    df_pos = df_pos.reset_index()
    df_prec_dev = pd.concat([df_prec_dev, df_pos.tail(1)], ignore_index = True)

  df_prec_dev = df_prec_dev.sort_values(by = 'Date', ascending=True)
  df_prec_dev.index = df_prec_dev['Date']

  df_prec_dev.drop(['Date'], axis=1, inplace=True)

  return df_prec_dev


# module to update my prospects
def update_my_prospects(df_tmp):
  mypps_df = read_csv('myProspectsScrips.csv')

  # Merge the two DataFrames on the common column
  merged_df = pd.merge(mypps_df, df_tmp, on='Symbol', how='left')

  # Update the column in the CSV DataFrame with the values from the second DataFrame
  mypps_df['Momentum'] = merged_df['Momentm']

  # Save the updated DataFrame back to a CSV file
  mypps_df.to_csv('data/myProspectsScrips.csv', index=False)
  df_tmp.to_csv('data/myProspects-Momentum.csv', index=False)

df_prec_dev = get_mypps_momentum(stock_ids)
df_tmp = df_prec_dev.drop(['Close'], axis=1)

update_my_prospects(df_tmp)

df_prec_dev.drop(['Momentm'], axis=1, inplace=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%******

In [3]:
df_prec_dev[df_prec_dev['Signal'] == 'Sell'].tail(20)

,Symbol,Close,Signal
Date,,,
2024-04-01,BERGEPAINT,509.0,Sell
2024-05-01,PGHH,15971.0,Sell
2024-05-01,TRIDENT,36.0,Sell
2024-05-01,VAIBHAVGBL,338.0,Sell
2024-05-01,EQUITASBNK,93.0,Sell
2024-05-02,UJJIVAN,591.0,Sell
2024-06-01,PARAGMILK,175.0,Sell
2024-06-01,TITAN,3404.0,Sell
2024-06-01,TV18BRDCST,41.0,Sell


In [4]:
df_prec_dev[df_prec_dev['Signal'] == 'Buy'].tail(20)

,Symbol,Close,Signal
Date,,,
2024-07-01,TTKPRESTIG,964.0,Buy
2024-07-01,HDFCLIFE,716.0,Buy
2024-07-01,VINATIORGA,2206.0,Buy
2024-07-01,PEL,1042.0,Buy
2024-07-01,INFY,1868.0,Buy
2024-07-01,PIIND,4431.0,Buy
2024-07-01,CLEAN,1622.0,Buy
2024-07-01,LUXIND,2065.0,Buy
2024-07-01,ROSSARI,834.0,Buy
